# Introduction

The process to annotate a text using one of the CRF models trained for the "Gelehrtenkorrespondenz" DFG project is articulated in the following steps:

1. Preprocess, which includes:
    * sentence tokenization
    * Part-Of-Speech tagging and lemmatization
2. Feature extraction
3. Annotation

What you need:
* Python 3
* Scikit learn
* NLTK
* TreeTagger and a Python TreeTagger wrapper

In [26]:
import nltk
import os
import pickle
import sys

In [28]:
sys.path.append("../")

In [29]:
from training import load_dictionaries
from templates import template1
from training import InstanceFeatureExtractor

/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/fmambrini/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [8]:
text = '''1847 kehrte Mommsen nach Deutschland zurück, musste vorerst aber wieder als Lehrer arbeiten. 
Während der Märzrevolution von 1848 wurde er Journalist in Rendsburg und vertrat energisch seine liberalen Überzeugungen. 
Im Herbst dieses Jahres erhielt er einen Ruf als außerordentlicher Professor für Rechtswissenschaft nach Leipzig und konnte so endlich die wissenschaftliche Laufbahn einschlagen. 
Er begann eine umfangreiche Publikationstätigkeit, blieb aber auch politisch aktiv, zusammen mit seinen Freunden und Professorenkollegen Moriz Haupt und Otto Jahn. Wegen ihrer Beteiligung am sächsischen Maiaufstand 1849 wurden die drei angeklagt und 1851 aus dem Hochschuldienst entlassen. 
Mommsen wurde noch im selben Jahr auf einen Lehrstuhl für Römisches Recht in Zürich berufen, den er 1852 antrat. Hier fühlte er sich jedoch sehr unwohl; er klagte in einem Brief über die Schweizer: „Die gehören zum Froschgeschlecht, und man muss Gott danken, wenn sie Hochdeutsch sprechen und eine Serviette auf den Tisch legen.“ Er wollte daher gern nach Deutschland zurückkehren und folgte 1854 einer Berufung nach Breslau, wo er mit dem Privatdozenten Jacob Bernays Freundschaft schloss. 1856 verlieh ihm die Philosophische Fakultät der Universität Greifswald die Ehrendoktorwürde. Allerdings gefiel Mommsen auch Breslau nicht; vor allem stießen ihn die dortigen Studenten ab: „Die meisten stinken, alle sind faul“'''

# Preprocessing

## Sentence tokenization

We use the German Punkt tokenizer that was trained on the iDAI Publication data set. Download it from [here]()

In [4]:
tok_path = '/Users/fmambrini/PycharmProjects/iDAIPublications/idai_journals/lib/nltk_data_extension/tokenizers/punkt/PY3/dai_german_punkt.pickle'

In [9]:
tokenizer = nltk.data.load(tok_path)
sents = tokenizer.tokenize(text)

## POS tagging and lemmatization

In [14]:
from treetagger import TreeTagger

In [15]:
tagged_sents = []
tt = TreeTagger(language='german')

for i, s in enumerate(sents):
    tags = tt.tag(s)
    tags = [tuple(tag + ["_", ""]) for tag in tags]
    tagged_sents.append(tags)

In [16]:
tagged_sents[0]

[('1847', 'CARD', '@card@', '_', ''),
 ('kehrte', 'VVFIN', 'kehren', '_', ''),
 ('Mommsen', 'NN', '<unknown>', '_', ''),
 ('nach', 'APPR', 'nach', '_', ''),
 ('Deutschland', 'NE', 'Deutschland', '_', ''),
 ('zurück', 'PTKVZ', 'zurück', '_', ''),
 (',', '$,', ',', '_', ''),
 ('musste', 'VMFIN', 'müssen', '_', ''),
 ('vorerst', 'ADV', 'vorerst', '_', ''),
 ('aber', 'ADV', 'aber', '_', ''),
 ('wieder', 'ADV', 'wieder', '_', ''),
 ('als', 'KOKOM', 'als', '_', ''),
 ('Lehrer', 'NN', 'Lehrer', '_', ''),
 ('arbeiten', 'VVINF', 'arbeiten', '_', ''),
 ('.', '$.', '.', '_', '')]

## Load the model

In [22]:
model_path = "../lib/models/korrespondez_model_stage3.pickle"

In [25]:
with open(model_path, 'rb') as f:
    crf = pickle.load(f)

# Feature extraction

## Load the dictionaries (places and persons)

In [31]:
d = {
    "persons": "../lib/dictionaries/persons.txt",
    "places": "../lib/dictionaries/places.txt"
  }

dicts = load_dictionaries(d)

In [32]:
dicts["persons"]

['Abate',
 'Pasquale',
 'Abeken',
 'Abeken',
 'Heinrich',
 'Abeken',
 'Abeken',
 'Wilhelm',
 'Acerbi',
 'Ackner',
 'Adam',
 'Frederick',
 'Adda',
 'Girolamo',
 'Adilardi',
 'Francesco',
 'Adler',
 'Friedrich',
 'Adorni',
 'Aegidi',
 'Aglietti',
 'Filippo',
 'Agostini',
 'Giosuè',
 'Ainsley',
 'James',
 'Alacevich',
 'Giuseppe',
 'Albiciri',
 'Cesare',
 'Albon',
 'Castillo',
 'Aldehoven',
 'Carl',
 'Aldini',
 'Pier Vittorio',
 'Alessi',
 'Giuseppe',
 'Almer',
 'Aloe',
 'Stanislao',
 'Alten',
 'Althoff',
 'Alvarez',
 'Anibal',
 'Amarelli',
 'Vincenzio',
 'Amari',
 'Amati',
 'Girolamo',
 'Ambrosch',
 'Julius Athanasius',
 'Ambrosi',
 'Ambrosio',
 'Vincenzo',
 'Amedei',
 'Amelung',
 'Walter',
 'Ammendola',
 'Giuseppe',
 'Ancona',
 'Amilcare',
 'Andrea',
 'Angelucci',
 'Giulio-Anastasio',
 'Angeluzzi',
 'Giuseppe',
 'Anglona',
 'Ankershofen',
 'Anselmi',
 'Ansidei',
 'Antaldi',
 'Antici-Mattei',
 'Antini',
 'Giuseppe',
 'Antonelli',
 'Agostino',
 'Antonelli',
 'Giuseppe',
 'Antonis',
 'Anto

## Run the feature extractor

In [34]:
ext = InstanceFeatureExtractor(tagged_sents, dicts)

In [36]:
%%time
X_anno = ext.extract_features(template1)

CPU times: user 19.8 ms, sys: 1.33 ms, total: 21.1 ms
Wall time: 20.7 ms


In [37]:
X_anno[0]

[{'BOS': 'True',
  'endsWithDigit[0]': 'True',
  'endsWithDigit[1]': 'False',
  'hasDigit[0]': 'True',
  'hasDigit[1]': 'False',
  'isDigit[0]': 'True',
  'isDigit[1]': 'False',
  'isInPersonDic[0]': 'False',
  'isInPersonDic[1]': 'False',
  'isInPersonDic[2]': 'True',
  'isInPlaceDic[0]': 'False',
  'isInPlaceDic[1]': 'False',
  'isInPlaceDic[2]': 'False',
  'isTitle[0]': 'False',
  'isTitle[1]': 'False',
  'isUpper[0]': 'False',
  'isUpper[1]': 'False',
  'lemma[0]': '@card@',
  'lemma[1]': 'kehren',
  'pos[0]': 'CARD',
  'pos[0]|pos[1]': 'CARD|VVFIN',
  'pos[0]|pos[1]|pos[2]': 'CARD|VVFIN|NN',
  'pos[1]': 'VVFIN',
  'pos[1]|pos[2]': 'VVFIN|NN',
  'pos[2]': 'NN',
  'prefix_long[0]': '1847',
  'prefix_long[1]': 'kehrt',
  'prefix_long[2]': 'Momms',
  'prefix_short[0]': '184',
  'prefix_short[1]': 'keh',
  'prefix_short[2]': 'Mom',
  'rank[0]': '0',
  'sent_rank[0]': '0',
  'suffix_long[0]': '1847',
  'suffix_long[0]|suffix_long[1]': '1847|hrte',
  'suffix_long[0]|suffix_long[1]|suffix

# Annotate 

In [38]:
y_pred = crf.predict(X_anno)

It's done! Now we just package everything together

In [42]:
annotation = []
for s, p in zip(tagged_sents, y_pred):
    anno_sent = []
    for tok_sent, tok_pred in zip(s, p):
        anno_sent.append(tuple(list(tok_sent[:-1]) + [tok_pred]))
    annotation.append(anno_sent)

**Et voillà...**

In [43]:
annotation[0]

[('1847', 'CARD', '@card@', '_', 'O'),
 ('kehrte', 'VVFIN', 'kehren', '_', 'O'),
 ('Mommsen', 'NN', '<unknown>', '_', 'B-PERmentioned'),
 ('nach', 'APPR', 'nach', '_', 'O'),
 ('Deutschland', 'NE', 'Deutschland', '_', 'B-PLACEmentioned'),
 ('zurück', 'PTKVZ', 'zurück', '_', 'O'),
 (',', '$,', ',', '_', 'O'),
 ('musste', 'VMFIN', 'müssen', '_', 'O'),
 ('vorerst', 'ADV', 'vorerst', '_', 'O'),
 ('aber', 'ADV', 'aber', '_', 'O'),
 ('wieder', 'ADV', 'wieder', '_', 'O'),
 ('als', 'KOKOM', 'als', '_', 'O'),
 ('Lehrer', 'NN', 'Lehrer', '_', 'O'),
 ('arbeiten', 'VVINF', 'arbeiten', '_', 'O'),
 ('.', '$.', '.', '_', 'O')]